In [2]:
import vk_api
import pandas as pd
import json
import tqdm

In [8]:

# Ваш access token
access_token = 'vk1.a.W7Dd87F9tTwraDI0JDCJrT4vZKNYVPTwIf8dZdLZMk5MqmTitG5jWgKhm1MnNq03vEl3LZHB9U23Ya0U05EEZj4B7EuGmg2JXvq0X3jEE-OlAgvbDyGfIdbLT0s-W8fu2e94b9BDX25vQt8BEy4moWQpGY5zKNx9G56gMKaZKUrbZb-n5aIQ7dwHXQ6Wkzrpui8qXx4q44b8Sv_1ACjdwg'

In [9]:

# Аутентификация
vk_session = vk_api.VkApi(token=access_token)

# Получение объекта API
vk = vk_session.get_api()

In [12]:
def get_posts(vk, owner_id, count=100):
    posts = []
    for i in range(0, count, 100):
        posts.extend(vk.wall.get(owner_id='-' + owner_id, count=100, offset=i)['items'])
    
    return posts

In [ ]:
def get_group_info(vk, group_id):
    return vk.groups.getById(group_id=group_id)[0]

In [14]:
groups = {
    'positive': {
        'science': [
            '151060688',
            '112289703',
            '99353432'
        ],   
        'art': [
            '2980565',
            '25813425',
            '211793398'
        ],
        'sport': [
            '30360552',
            '217808619',
            '47610600'
        ]
    },
    'negative': {
        'depression': [
            '180719396',
            '212525572',
            '203151176'
            
        ],   
        'aggression': [
            '205871237',
            '195771683',
            '45799824'
        ],
        'trash': [
            '23148107',
            '173718071',
            '170218556'
        ]
    }
}

In [ ]:
all_posts = []

for tone in groups:
    for topic in groups[tone]:
        for group_id in groups[tone][topic]:
            print(group_id)
            posts = get_posts(vk, group_id, count=100)
            for post in posts:
                all_posts.append({
                    'text': post['text'],
                    'tone': tone,
                    'topic': topic,
                    'group_id': group_id,
                    'comments_cnt:': post['comments']['count'],
                    'likes_cnt': post['likes']['count'],
                    'reposts_cnt': post['reposts']['count'],
                    'from_id': post['from_id'],
                    'post_id': post['id'],
                    'views_cnt': post['views']['count'] if 'views' in post else 1,
                })

In [50]:
with open('data/posts.json', 'w') as f:
    json.dump(all_posts, f)

In [51]:
posts_data = pd.json_normalize(all_posts)

In [48]:
#posts_data.to_csv('data/posts.csv')
posts_data = pd.read_csv('data/posts.csv')

In [ ]:
posts_data

In [ ]:
all_comments = []

for rn, row in tqdm.tqdm(posts_data.iterrows(), total=posts_data.shape[0]):
    try:
        comments = vk.wall.getComments(owner_id=row['from_id'], post_id=row['post_id'], count=100)  # Пример 
    except:
        print(row['from_id'], row['post_id'])
        continue
    for comment in comments['items']:
        all_comments.append(comment)

In [51]:
with open('data/comments.json', 'w') as f:
    json.dump(all_comments, f)

In [ ]:
all_comments

In [15]:
posts = get_posts(vk, "25813425", count=10)

In [ ]:
posts[0]['likes']

## Get users

In [10]:

posts = pd.read_csv('data/posts.csv')
with open('data/comments.json', 'r') as f:
    comments = json.load(f)

In [11]:
users_ids = set()
for comment in comments:
    users_ids.add(comment['from_id'])
    
for user_id in posts['from_id']:
    users_ids.add(user_id)

In [ ]:
len(users_ids)

In [23]:
users = []

for i in range(0, len(users_ids), 100):
    user_id_str = ','.join([str(user_id) for user_id in list(users_ids)[i:i+100]])
    try:
        users_new = vk.users.get(user_ids=user_id_str, fields=['bdate', 'sex'])
    except Exception as e:
        print(e)
        break
    users.extend(users_new)

In [39]:
yang_users = []

for user in users:
    if 'bdate' in user and len(user['bdate'].split('.')) == 3:
        if 2005 > int(user['bdate'].split('.')[2]) > 1987:
            yang_users.append(user)

In [ ]:
len(yang_users)

In [ ]:
len(users)

In [41]:
users_data = pd.json_normalize(yang_users)

In [42]:
users_data[['id', 'bdate', 'sex']].to_csv('data/users.csv', index=False)